# Virtual Piano Tutor

## Project Description

This is a notebook for tracking my progress on VPT...

- Best Classifier as of 11/30
    - SVM {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}

## TODO List

- TODAY
    - Decide on RDF model to keep for rest of project
    - Work on RDF data and annotations
    - Add results to file
    - Rewrite RDF for GridSearchCV
        - Extend RDF
    - Work on ideas for paper
        - Visualizations
    - Play with CAE
    - How to automate this...
    - Windowing/Summarizing
    
- DONE
    - ~~Organize RDF data~~
    - ~~Generate data from already extracted hands...~~
    - ~~Get notebook running on Compute Canada~~
    - ~~Get data on Compute Canada~~
    - ~~Setup CAE to deal with hand images~~
    - ~~setup data for training autoencoder on LH and RH~~
    - ~~Train Autoencoder for LH and Rh~~
    

- Bad Segmentation
    - p3c - left hand (not terrible)
    - p1s - right hand (shouldn't use)
    - p5a - Both could use some work but still caputures most of the left hand (RH not so good...)
    - p5c - not good (left hand passable...)
    
- Add noise to CAE
    - http://scikit-image.org/docs/dev/api/skimage.util.html#random-noise
    
- ~~Multiple Participants~~
    - ~~have one holdout set participant~~
        - ~~Test with p1&2 training p3 testing, then p1&3...~~
    - ~~have one holdout set exercise~~

- Test with RH too

- Windowing data
    - Summarize data for classification
    - Majority Voting (or with probabilities)

- Look for other features
    - Others??
    - ~~Autoencoder features~~
    - ~~HONV~~
    
- Work on hand segmentation
   - See p1e for bad examples
   - How to validate segmentation?
       - Statistical analysis on length and width ratios
       
- Visualize !!!
    - Input 
    - Results !!!
        - F Scores
        - Accuracy
        - Try weighted instead of macro




- Finish Project Description

- ~~Turn into functions~~
    
- ~~Verify Segmentation~~
    - have only done basic verification
    
- ~~FIRST THING: Test by ignoring training data (p1s) and then using train_test_split on recordings~~
    - ~~Data should be ready for spliting~~
    
- ~~Remove data from testing to find culprit~~
    
- ~~Track my progress better !!! (duh through notebooks!)~~

# Setup

## Libraries

In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s
import vpt.hand_detection.depth_context_features as dcf

%load_ext autoreload
%autoreload 2

## Some helper functions

### Load/Save Data Set

In [2]:
def load_data(testing_p, M, radius, feature_type, data_type):
    base = "data/posture/extracted/"
    data_path = os.path.join(base, "{}_M{}_rad{:0.2f}_{}_".format(testing_p, M, radius, feature_type))
    data = np.load(data_path + data_type + "_data_combined.npz")    
    return data

## Project Setup

### Generate or Load Data

In [3]:
M = 5
radius = .15
feature_type = "hog"
testing_p = "p3"

In [4]:
#### Load data for a single paricipant
data = load_data("all_participants", M, radius, feature_type, "train")
print("X LH", data["X_lh"].shape, "y LH", data["y_lh"].shape, data["vis_lhs"].shape)
print("X RH", data["X_rh"].shape, "y RH", data["y_rh"].shape, data["vis_rhs"].shape)
print("Filenames", data["filenames"].shape)

X LH (15818, 1089) y LH (15818,) (15818, 180, 180)
X RH (15818, 1089) y RH (15818,) (15818, 180, 180)
Filenames (15818,)


In [5]:
## using p6 for validation
r = re.compile(testing_p)
vmatch = np.vectorize(lambda x:bool(r.search(x)))
val_p = vmatch(data['filenames'])

X_lh = data['X_lh'][~val_p]
y_lh = data['y_lh'][~val_p]
X_rh = data['X_rh'][~val_p]
y_rh = data['y_rh'][~val_p]
filenames = data['filenames'][~val_p]


X_lh_test = data['X_lh'][val_p]
y_lh_test = data['y_lh'][val_p]
X_rh_test = data['X_rh'][val_p]
y_rh_test = data['y_rh'][val_p]
filenames_test = data['filenames'][val_p]

print("Cross Val Data:")
print(X_lh.shape, y_lh.shape)
print(X_rh.shape, y_rh.shape)
print(filenames.shape)
print()
print("Validation Data")
print(X_lh_test.shape, y_lh_test.shape)
print(X_rh_test.shape, y_rh_test.shape)
print(filenames_test.shape)

Cross Val Data:
(11246, 1089) (11246,)
(11246, 1089) (11246,)
(11246,)

Validation Data
(4572, 1089) (4572,)
(4572, 1089) (4572,)
(4572,)


In [6]:
groups = np.zeros_like(filenames, dtype=int)

for p in ["p1", "p3", "p4", "p6"]:
    p_num = int(p[1])
    groups[np.where(np.char.find(filenames, p) != -1)] = p_num
    
print(groups.shape)
print(np.unique(groups))
print(groups)

(11246,)
[1 4 6]
[1 1 1 ..., 6 6 6]


# Classification

### Libraries

In [24]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, LeaveOneGroupOut, GroupKFold
from imblearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours

## Load Data for Classification

In [8]:
## find all "static" data so we can ignore for now
r = re.compile('p[\d]s')

# remove p#s data
vmatch = np.vectorize(lambda x:bool(r.search(x)))
rem_static = vmatch(filenames)
rem_static_test = vmatch(filenames_test)

X_lh_train, y_lh_train, filenames_train, groups_train = X_lh[~rem_static], y_lh[~rem_static], filenames[~rem_static], groups[~rem_static]
X_rh_train, y_rh_train, filenames_train, groups_train = X_rh[~rem_static], y_rh[~rem_static], filenames[~rem_static], groups[~rem_static]

In [9]:
X_lh_val, y_lh_val, filenames_val = X_lh_test[~rem_static_test], y_lh_test[~rem_static_test], filenames_test[~rem_static_test]
X_rh_val, y_rh_val, filenames_val = X_rh_test[~rem_static_test], y_rh_test[~rem_static_test], filenames_test[~rem_static_test]

In [10]:
X_comb_train = np.vstack((X_lh_train, X_rh_train))
y_comb_train = np.hstack((y_lh_train, y_rh_train))
groups_comb_train = np.hstack((groups_train, groups_train))
filenames_comb_train = np.hstack((filenames_train, filenames_train))

X_comb_val = np.vstack((X_lh_val, X_rh_val))
y_comb_val = np.hstack((y_lh_val, y_rh_val))
filenames_comb_val = np.hstack((filenames_val, filenames_val))

In [11]:
print("Cross Validation Data")
print("LH:", X_lh_train.shape, y_lh_train.shape)
print(np.unique(y_lh_train, return_counts=True))
print("RH:", X_rh_train.shape, y_rh_train.shape)
print(filenames_train.shape)
print(groups_train.shape)
print(np.unique(y_rh_train, return_counts=True))
print("Combined:", X_comb_train.shape, y_comb_train.shape)
print(filenames_comb_train.shape)
print(groups_comb_train.shape)
print(np.unique(y_comb_train, return_counts=True))
print()
print()
print("Validation Data")
print("LH:", X_lh_val.shape, y_lh_val.shape)
print(np.unique(y_lh_val, return_counts=True))
print("RH:", X_rh_val.shape, y_rh_val.shape)
print(np.unique(y_rh_val, return_counts=True))
print("Combined:", X_comb_val.shape, y_comb_val.shape)
print(np.unique(y_comb_val, return_counts=True))
print(filenames_comb_val.shape)

Cross Validation Data
LH: (9629, 1089) (9629,)
(array([0, 1, 2]), array([6890, 1718, 1021]))
RH: (9629, 1089) (9629,)
(9629,)
(9629,)
(array([0, 1, 2]), array([7496, 2106,   27]))
Combined: (19258, 1089) (19258,)
(19258,)
(19258,)
(array([0, 1, 2]), array([14386,  3824,  1048]))


Validation Data
LH: (2650, 1089) (2650,)
(array([0, 1, 2]), array([1267, 1336,   47]))
RH: (2650, 1089) (2650,)
(array([0]), array([2650]))
Combined: (5300, 1089) (5300,)
(array([0, 1, 2]), array([3917, 1336,   47]))
(5300,)


In [18]:
y_comb_train_bin = np.zeros_like(y_comb_train)
y_comb_train_bin[y_comb_train != 0] = 1

y_comb_val_bin = np.zeros_like(y_comb_val)
y_comb_val_bin[y_comb_val != 0] = 1

print(np.unique(y_comb_train_bin, return_counts=True))
print(np.unique(y_comb_val_bin, return_counts=True))

(array([0, 1]), array([14386,  4872]))
(array([0, 1]), array([3917, 1383]))


## Model Testing

### SVM

In [31]:
size = 30
groups_kfold = np.zeros_like(filenames_comb_train, dtype=int)
for i in range(len(filenames_comb_train) // size):
    groups_kfold[i * size:i * size + size] = i
groups_kfold[len(groups_kfold) % size * -1:] = i + 1

cv = GroupKFold(n_splits=3)

In [32]:
## Parameters for SVMs
steps = [('SMOTE', SMOTEENN()), ("SVC", SVC())]
# param_grid = [
# #   {'SVC__C': [1, 10, 100], 'SVC__kernel': ['linear'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
# #   {'SVC__C': [1, 10, 100], 'SVC__gamma': [.0001, .001, .01, .1], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2', 'svm']},
#   {'SVC__C': [10, 100], 'SVC__gamma': [.000005, .00001, .00005,], 'SVC__kernel': ['rbf'], 'SMOTE__kind': ['regular', 'borderline1', 'borderline2']},
#  ]

steps = [('SMOTE', SMOTEENN()), ("SVC", SVC())]
param_grid = [
  {'SVC__C': [1, 10, 100], 
   'SVC__kernel': ['linear'], 
   'SMOTE__smote': [SMOTE(kind='borderline2')], 
   "SMOTE__enn": [EditedNearestNeighbours(n_neighbors=7, kind_sel='mode'), EditedNearestNeighbours(n_neighbors=5, kind_sel='mode')]},
  {'SVC__C': [1, 10, 100], 
   'SVC__gamma': [.001, .01, .1], 
   'SVC__kernel': ['rbf'], 
   'SMOTE__smote': [SMOTE(kind='borderline2')], 
   "SMOTE__enn": [EditedNearestNeighbours(n_neighbors=7, kind_sel='mode'), EditedNearestNeighbours(n_neighbors=5, kind_sel='mode')]},
 ]


pipeline = Pipeline(steps)

scores = ['accuracy']
logo = GroupKFold()

In [33]:
# Hyper Parameter Tuning
for score in scores:
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        print("## Tuning hyper-parameters for {}".format(score))
        print()

        if score is "accuracy":
            scoring = score
        else:
            scoring = '{}_weighted'.format(score)      
        
        #### TRAIN COMBINED LH & RH
        clf_comb = GridSearchCV(pipeline, param_grid, cv=logo.split(X_comb_train, y_comb_train_bin,groups=groups_kfold), scoring=scoring, n_jobs=2, verbose=10)
        clf_comb.fit(X_comb_train, y_comb_train_bin)

        print("Best Combined Parameters set found on data set:")
        print()
        print(clf_comb.best_params_)
        print()
        print("Grid scores on data set:")
        print()
        means = clf_comb.cv_results_['mean_test_score']
        stds  = clf_comb.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_comb.cv_results_['params']):
            print("%0.3f (+/-%0.3f) for %r" % (mean, std, params))
        print()

## Tuning hyper-parameters for accuracy

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=1, SVC__kernel=linear 
[CV] SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=1, SVC__kernel=linear 
[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=F

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:  9.7min


[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=1, SVC__kernel=linear, score=0.8557632398753894, total= 9.3min
[CV] SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=10, SVC__kernel=linear 
[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None,

[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed: 29.3min


[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=10, SVC__kernel=linear, score=0.8436137071651091, total=10.4min
[CV] SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=100, SVC__kernel=linear 
[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=Non

...

   svm_estimator=None), SVC__C=100, SVC__gamma=0.01, SVC__kernel=rbf 
[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=5,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=100, SVC__gamma=0.01, SVC__kernel=rbf, score=0.9213395638629284, total= 7.8min
[CV] SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=5,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf 


[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed: 537.0min


[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=5,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=100, SVC__gamma=0.01, SVC__kernel=rbf, score=0.9035525085696479, total= 8.3min
[CV] SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=5,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), SMOTE__smote=SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), SVC__C=100, SVC__gamma=0.1, SVC__kernel=rbf 
[CV]  SMOTE__enn=EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=5,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=Non

[Parallel(n_jobs=2)]: Done  72 out of  72 | elapsed: 571.8min finished


Best Combined Parameters set found on data set:

{'SMOTE__enn': EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=5,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), 'SMOTE__smote': SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), 'SVC__C': 100, 'SVC__gamma': 0.1, 'SVC__kernel': 'rbf'}

Grid scores on data set:

0.848 (+/-0.006) for {'SMOTE__enn': EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
            random_state=None, ratio='auto', return_indices=False,
            size_ngh=None), 'SMOTE__smote': SMOTE(k=None, k_neighbors=5, kind='borderline2', m=None, m_neighbors=10,
   n_jobs=1, out_step=0.5, random_state=None, ratio='auto',
   svm_estimator=None), 'SVC__C': 1, 'SVC__kernel': 'linear'}
0.846 (+/-0.006) for {'SMOTE__enn': EditedNearestNeighbours(kind_sel='mode', n_jobs=1, n_neighbors=7,
         

In [34]:
from sklearn.metrics import accuracy_score
y_comb_true, y_comb_pred = y_comb_val_bin, clf_comb.predict(X_comb_val)

print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))
print()


Comb Validatation Score: 0.826981132075
Comb Confustion Matrix:
 [[3888   29]
 [ 888  495]]
             precision    recall  f1-score   support

          0       0.81      0.99      0.89      3917
          1       0.94      0.36      0.52      1383

avg / total       0.85      0.83      0.80      5300




### Single Classifier

In [ ]:
# steps = [('PCA', PCA(n_components=1500)), ('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=100, gamma=.0001, kernel='rbf', probability=False))]
steps = [('SMOTE', SMOTE(kind="regular")), ("SVC", SVC(C=100, gamma=.00005, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
pipeline.fit(X_comb_train, y_comb_train)

In [ ]:
y_comb_true, y_comb_pred = y_comb_val, pipeline.predict(X_comb_val)
print("Comb Validatation Score:", accuracy_score(y_comb_true, y_comb_pred))
print("Comb Confustion Matrix:\n", confusion_matrix(y_comb_true, y_comb_pred))
print(classification_report(y_comb_true, y_comb_pred))

In [47]:
window_size = 5
y_comb_true_maj = []
for i in range(0, y_comb_true.size, window_size):
    u, counts = np.unique(y_comb_true[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_true_maj.append(pred)
    
y_comb_true_maj = np.array(y_comb_true_maj)

In [48]:
y_comb_pred_maj = []
for i in range(0, y_comb_pred.size, window_size):
    u, counts = np.unique(y_comb_pred[i:i+window_size], return_counts=True)
    pred = u[np.argmax(counts)]
    y_comb_pred_maj.append(pred)
    
y_comb_pred_maj = np.array(y_comb_pred_maj)

In [49]:
print("rH Validatation Score:", accuracy_score(y_comb_true_maj, y_comb_pred_maj))
print("rH Confustion Matrix:\n", confusion_matrix(y_comb_true_maj, y_comb_pred_maj))
print(classification_report(y_comb_true_maj, y_comb_pred_maj))

rH Validatation Score: 0.831132075472
rH Confustion Matrix:
 [[780   3]
 [176 101]]
             precision    recall  f1-score   support

          0       0.82      1.00      0.90       783
          1       0.97      0.36      0.53       277

avg / total       0.86      0.83      0.80      1060



### Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
X_comb_all_cv = np.vstack((X_comb_train, X_comb_val))
y_comb_all_cv = np.hstack((y_comb_train, y_comb_val))
filenames_comb_all_cv = np.hstack((filenames_comb_train, filenames_comb_val))

print("Combined:", X_comb_all_cv.shape, y_comb_all_cv.shape)
print(np.unique(y_comb_all_cv, return_counts=True))
print(filenames_comb_all_cv.shape)

In [ ]:
groups_cv = np.zeros_like(filenames_comb_all_cv, dtype=int)

for p in ["p1", "p3", "p4", "p6"]:
    p_num = int(p[1])
    groups_cv[np.where(np.char.find(filenames_comb_all_cv, p) != -1)] = p_num
    
print(groups_cv.shape)
print(np.unique(groups_cv))
print(groups_cv)

In [ ]:
steps = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
scoring = ("f1_macro", "accuracy")
logo = LeaveOneGroupOut()
scores = cross_validate(pipeline, X_comb_all_cv, y_comb_all_cv, cv=logo.get_n_splits(groups=groups_cv), scoring=scoring, n_jobs=2, verbose=2)

In [ ]:
print(scores)

In [ ]:
for k, v in scores.items():
    print(k, v)

In [ ]:
steps = [('SMOTE', SMOTE(kind="borderline2")), ("SVC", SVC(C=1, gamma=.001, kernel='rbf', probability=False))]
pipeline = Pipeline(steps)
scoring = ("f1_macro", "accuracy")
logo = LeaveOneGroupOut()
scores = cross_validate(pipeline, X_comb_all_cv, y_comb_all_cv, cv=logo.split(X_comb_all_cv, y_comb_all_cv, groups=groups_cv), scoring=scoring, n_jobs=2, verbose=3)

In [ ]:
for k, v in scores.items():
    print(k, v)

In [ ]:
logo.split(X_comb_all_cv, y_comb_all_cv, groups=groups_cv)